In [ ]:
import pandas as pd
from scipy.stats import zscore
import os

# Пути к исходным файлам
combined_scores_path = r"../../depmap_datasets/prepared_datasets/depscores.csv"
merged_data_path = r"../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv"

# Папка для сохранения результатов
output_folder = r"./mean_z-scores_by_cancer_type/"

# Чтение файлов
df_combined = pd.read_csv(combined_scores_path)
df_merged = pd.read_csv(merged_data_path)

# Фильтрация клеточных линий по заболеваниям
valid_diseases = df_merged['primary_disease_renamed'].value_counts().index.tolist()
df_valid_lines = df_combined[df_combined['DepMap_ID'].isin(df_merged[df_merged['primary_disease_renamed'].isin(valid_diseases)]['ModelID'])]

# Добавление типа заболевания
df_valid_lines = pd.merge(df_valid_lines, df_merged[['ModelID', 'primary_disease_renamed']], left_on='DepMap_ID', right_on='ModelID', how='left')

# Вычисление Z-оценки для числовых столбцов (кроме идентификаторов)
numeric_cols = df_valid_lines.columns[1:-2]
df_zscore = df_valid_lines.copy()
df_zscore[numeric_cols] = df_valid_lines[numeric_cols].apply(zscore, nan_policy='omit')

# Фильтрация и вычисление средних значений по выбранным типам опухолей
selected_types = ["CML", "CLL", "AML", "ALL", "Ovarian Cancer", "Neuroblastoma", "Colon/Colorectal Cancer", "Lung Cancer"]
df_filtered = df_zscore[df_zscore["primary_disease_renamed"].isin(selected_types)]
df_mean = df_filtered.groupby("primary_disease_renamed").mean(numeric_only=True)

# Бинаризация Z-оценок для каждого типа рака
thresholds = [-0.5, -1, -1.5, -2, -2.5, -3]

# Создание папки для бинарных файлов, если её нет
os.makedirs(output_folder, exist_ok=True)

for cancer_type in df_mean.index:
    cancer_series = df_mean.loc[cancer_type]
    
    # Создаём DataFrame, где для каждого порога вычисляем бинарное значение для каждого столбца
    cancer_binary_df = pd.DataFrame({
        str(threshold): (cancer_series <= threshold) for threshold in thresholds
    })
    
    # Преобразуем True/False в "Да"/"Нет"
    cancer_binary_df = cancer_binary_df.astype(str).replace({"True": "Да", "False": "Нет"})
    
    # Рассчитываем процентное соотношение "Да"/"Нет" для каждого столбца
    percent_df = cancer_binary_df.apply(lambda x: (x == "Да").mean() * 100, axis=0)
    
    # Добавляем строку "Процент"
    cancer_binary_df.loc["Процент"] = percent_df
    
    # Переставляем последнюю строку на первое место
    cancer_binary_df = pd.concat([cancer_binary_df.iloc[[-1]], cancer_binary_df.iloc[:-1]])

    # Сохраняем каждый файл для типа рака
    output_path = f"{output_folder}\\mean_z-score_{cancer_type.replace('/', '_').lower()}.csv"
    cancer_binary_df.to_csv(output_path)
    print(f"Файл для {cancer_type} сохранён: {output_path}")

print("Все бинарные файлы успешно сохранены!")

Файл для ALL сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_all.csv
Файл для AML сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_aml.csv
Файл для CLL сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_cll.csv
Файл для CML сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_cml.csv
Файл для Colon/Colorectal Cancer сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_colon_colorectal cancer.csv
Файл для Lung Cancer сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_lung cancer.csv
Файл для Neuroblastoma сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_neuroblastoma.csv
Файл для Ovarian Cancer сохранён: ./mean_z-scores_by_cancer_type/\mean_z-score_ovarian cancer.csv
Все бинарные файлы успешно сохранены!
